In [14]:
import sys
import yaml
import os
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
from config.notebook_location import find_project_root

In [16]:
# Definir el nombre del directorio del proyecto
project_name = "personal-library"

# Encontrar la raíz del proyecto
project_root = find_project_root(project_name)

# Agregar la raíz del proyecto al principio de sys.path para facilitar las importaciones
sys.path.insert(0, project_root)
print("Project root added to sys.path:", project_root)

Project root added to sys.path: d:\Espacios de trabajo\personal-library


In [17]:
from ingestion.sources.from_api import APIDatasetLoader
from ingestion.governance.engine import GovernanceEngine
from ingestion.governance.engine_policy_autogen import get_or_create_policy
from ingestion.governance.metrics import MetricsEngine
from ingestion.base.metadata_logger import MetadataLogger
from ingestion.governance.remediation import RemediationEngine
from ingestion.governance.intelligent_improvement import IntelligentImprovementEngine

In [18]:
# Inicializar el logger de metadatos
meta_logger = MetadataLogger(report_path="demos/reports/demo_here_api.parquet")

In [19]:
# Creamos la instancia del loader para Socrata.
loader = APIDatasetLoader()

In [20]:
params = {
    'apikey': os.getenv("HERE_API_KEY"),
    'q': 'empresas de software',
    'at': '4.60971,-74.08175'
}

In [21]:
meta_logger.log({
    "step": "data_extraction",
    "source": "here_api",
    "dataset_code": "",
    "filters": f"'q': {params.get('q')}, 'at': {params.get('at')}",
    "limit": "100",
    "status": "started"
})

In [22]:
df = loader.load_data(
    url = 'https://discover.search.hereapi.com/v1/discover',
    method = "GET",
    headers = None,
    params = params,
    body = None,
    body_type = "json",  # json, form, raw
    json_path = 'items', # Campo donde están los datos
    timeout = 15,
    pagination = None,
    max_pages = 10,
    disaggregate=True
)

In [23]:
meta_logger.log({
    "step": "data_extraction",
    "record_count": len(df),
    "status": "completed"
})

In [24]:
policy = get_or_create_policy(df, "here.yaml")

In [25]:
# Imprimimos la política cargada para verificar su contenido.
print("Política de Gobernanza Cargada:")
print(yaml.dump(policy, sort_keys=False, allow_unicode=True))

Política de Gobernanza Cargada:
dataset_metadata:
  columns: 38
  rows_sampled: 5
  generated_by: default_policy_generator
  generated_on: '2025-03-31T18:18:58.461324Z'
  data_source: inferred
  schema_version: '1.0'
  language: undetected
compliance:
  compliance_frameworks:
  - GDPR
  risk_level: high
enforcement_requirements:
  mandatory_rules:
  - no_nulls
  - valid_date_format
  required_fields: []
  privacy_enforcement: medium
  security_baseline: masked
  allowed_transparency:
  - internal
  - public
  risk_acceptance: medium
  framework_enforcement:
  - GDPR
fields:
- field_name: title
  type: string
  required: true
  rules: []
  privacy_level: low
  security: none
  transparency: internal
  integrity:
    unique: true
    no_nulls: true
    consistent_format: true
    contains_outliers: false
  compliance_tags: []
  data_subject: false
  access_restriction: public
  retention_policy: default
  critical_field: false
  justification: Field 'title' classified as string with priv